In [2]:
import numpy as np
import pandas as pd
import os
from itertools import product

In [3]:
mon_lst = []
for y in range(2000, 2025):
    for m in ['01-31', '02-29' if (y % 4 == 0 and y % 100 != 0) or (y % 400 == 0) else '02-28', '03-31', '04-30', '05-31', '06-30', '07-31', '08-31', '09-30', '10-31', '11-30', '12-31']:
        mon_lst.append(f'{y}-{m}')
mon_df = pd.DataFrame(mon_lst[:-2], columns=['交易月份'])
mon_df['月份'] = mon_df['交易月份'].apply(lambda x: x[:7])
stk_df = pd.read_csv('公司文件.csv')
stk_df['on'] = 0
mon_df['on'] = 0
header = pd.merge(stk_df, mon_df, on=['on'], how='left')[['证券代码', '交易月份', '月份']]
header

,证券代码,交易月份,月份
0,1,2000-01-31,2000-01
1,1,2000-02-29,2000-02
2,1,2000-03-31,2000-03
3,1,2000-04-30,2000-04
4,1,2000-05-31,2000-05
...,...,...,...
1719157,920118,2024-06-30,2024-06
1719158,920118,2024-07-31,2024-07
1719159,920118,2024-08-31,2024-08
1719160,920118,2024-09-30,2024-09


In [6]:
import numpy as np 
import pandas as pd 

RET = pd.read_csv('日_日个股回报率文件.csv')
RET = RET[['证券代码', '交易日期','考虑现金红利再投资的日个股回报率']]

def My_YM(Set,Col):
    Set['DATE'] = Set[Col].astype(str).replace('\-', '', regex=True)
    Set['Yearmon'] = Set['DATE'].astype(int) // 100

My_YM(RET,'交易日期')

#%%
RETS = RET[['考虑现金红利再投资的日个股回报率', '证券代码','Yearmon']
         ].groupby(['证券代码','Yearmon']
         ).std().reset_index()
RETS['volatility'] = RETS['考虑现金红利再投资的日个股回报率'].shift(1)
RETS = RETS.dropna()
RETS['月份'] = RETS['Yearmon'].astype(str).str[:4] + '-' + RETS['Yearmon'].astype(str).str[4:6]

# 最终选择的列
RETS = RETS[['证券代码', '月份', 'volatility']]

#  header 已经定义并包含 '证券代码' 和 '月份' 列
df = pd.merge(header, RETS, on=['证券代码', '月份'], how='left')

df = df[['证券代码','交易月份','volatility']]
df.to_csv("月_68.csv", encoding='utf_8_sig',index = False)

<>:8: SyntaxWarning: invalid escape sequence '\-'
<>:8: SyntaxWarning: invalid escape sequence '\-'
/tmp/ipykernel_8222/531672509.py:8: SyntaxWarning: invalid escape sequence '\-'
  Set['DATE'] = Set[Col].astype(str).replace('\-', '', regex=True)


In [7]:
df

,证券代码,交易月份,volatility
0,1,2000-01-31,NaN
1,1,2000-02-29,0.028339
2,1,2000-03-31,0.038460
3,1,2000-04-30,0.025564
4,1,2000-05-31,0.012733
...,...,...,...
1719157,920118,2024-06-30,NaN
1719158,920118,2024-07-31,NaN
1719159,920118,2024-08-31,0.098403
1719160,920118,2024-09-30,0.158275
